In [11]:
import pandas as pd
df = pd.read_csv('IBM.csv')

import numpy as np
from tqdm import tqdm_notebook as tqdm
def create_labels(df, col_name, window_size=11):
        row_counter = 0
        total_rows = len(df)
        labels = np.zeros(total_rows)
        labels[:] = np.nan
        print("Calculating labels")
        pbar = tqdm(total=total_rows)

        while row_counter < total_rows:
            if row_counter >= window_size - 1:
                window_begin = row_counter - (window_size - 1)
                window_end = row_counter
                window_middle = int((window_begin + window_end) / 2)

                min_ = np.inf
                min_index = -1
                max_ = -np.inf
                max_index = -1
                for i in range(window_begin, window_end + 1):
                    price = df.iloc[i][col_name]
                    if price < min_:
                        min_ = price
                        min_index = i
                    if price > max_:
                        max_ = price
                        max_index = i

                if max_index == window_middle:
                    labels[window_middle] = 0
                elif min_index == window_middle:
                    labels[window_middle] = 1
                else:
                    labels[window_middle] = 2

            row_counter = row_counter + 1
            pbar.update(1)

        pbar.close()
        return labels

y = create_labels(df, 'Close(t)', 11)

Calculating labels


<ipython-input-11-33094377dee2>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  pbar = tqdm(total=total_rows)


  0%|          | 0/4988 [00:00<?, ?it/s]

In [15]:
tempdf = pd.DataFrame(y, columns = ['labels'])
tempdf.to_csv('labels.csv')

In [16]:
df.drop(df.tail(5).index,inplace = True)
df.drop(df.head(4).index,inplace = True)
df = df.reset_index()

df2 = pd.read_csv('labels.csv')
df2.drop(df2.tail(5).index,inplace = True)
df2.drop(df2.head(4).index,inplace = True)
df3 = df2['labels']

df3 = df3.reset_index()
df3 = df3.drop(['index'], axis=1)
df = df.drop(['index'], axis=1)

df = df.join(df3)
df.to_csv('IBM_cleaned.csv')